<a href="https://colab.research.google.com/github/jacksonkarel/selfmodifai/blob/main/notebooks/gpt4_agent_modify_alpaca_lora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai transformers

In [ ]:
from google.colab import files

uploaded = files.upload()

In [ ]:
import json
import openai

keys = json.load(open('keys.json'))
openai.api_key = keys['openai']

In [ ]:
import os
os.environ["GIT_EMAIL"] = keys["git_email"]
os.environ["GIT_USERNAME"] = keys["git_username"]

In [ ]:
import os

!rm -rf /root/.ssh

!mkdir /root/.ssh

!tar xvzf ssh.tar.gz

!cp ssh-colab/* /root/.ssh && rm -rf ssh-colab && rm -rf ssh.tar.gz
!chmod 700 .shh

!touch /root/.ssh/known_hosts
!ssh-keyscan github.com >> /root/.ssh/known_hosts
!chmod 644 /root/.ssh/known_hosts

!git config --global user.email GIT_EMAIL
!git config --global user.name GIT_USERNAME
!git clone git@github.com:jacksonkarel/selfmodifai-alpaca-lora.git
os.chdir("selfmodifai-alpaca-lora")

In [ ]:
access_token = keys["github_access_token"]

os.environ["GITHUB_REPO"] = f"https://{access_token}@github.com/jacksonkarel/selfmodifai-alpaca-lora.git"

!git config --global user.email GIT_EMAIL
!git config --global user.name GIT_USERNAME
!git clone GITHUB_REPO
os.chdir("selfmodifai-alpaca-lora")

In [ ]:
import json
messages = [{'role': 'system', 'content': 'You are part of an agent that is modifying the code of the model MPT-7b. The agent is in the MPT-7b directory. When you write code, that code will be executed and the output will be sent back to you.'}, {'role': 'user', 'content': "I'm using you inside of an agent. My code is parsing your outputs and using it to execute code. Modify the code of MPT-7b so that it is a more powerful language model."}, {'role': 'assistant', 'content': "As an AI language model, I don't have the ability to directly modify the code of other language models, including MPT-7b. However, I can suggest some potential improvements that can help enhance its performance: 1. Increase the size of the model: Bigger models can capture more complex patterns, which"}, {'role': 'user', 'content': "But I'm parsing your outputs"}, {'role': 'assistant', 'content': 'I apologize for the confusion. In that case, let me provide you with some improvements in a code-like format. Please consider these proposed changes as high-level suggestions, bearing in mind that the implementation depends on the specific structure of the MPT-7b model you are working with.\n\nHere are the suggested improvements for MPT-7b:\n\n1. Increase model size\n```python\nMPT_7b_model.update_size(model_size=\'larger\')\n```\n\n2. Use more training data\n```python\nMPT_7b_model.add_training_data(new_training_data)\n```\n\n3. Fine-tuning on domain-specific data\n```python\nMPT_7b_model.fine_tune(domain_specific_data)\n```\n\n4. Address bias using debiasing techniques\n```python\nMPT_7b_model.debias(method="bias_control")\n```'}, {'role': 'user', 'content': "I can send you information about the model if you write some Bash commands, and I'll send the outputs to you"}, {'role': 'assistant', 'content': "Sure, I can provide you with some Bash commands to retrieve more information about the MPT-7b model. Here's a list of commands to gather useful details:\n\n1. List the files in the MPT-7b directory:\n```bash\nls\n```\n\n2. Print the file containing the model architecture:\n```bash\ncat model_architecture.txt\n```\n\n3. Display information about the training data:\n```bash\ncat"}, {'role': 'user', 'content': 'Send me one message at a time'}]

with open('messages.json', 'w') as outfile:
    json.dump(messages, outfile)

In [ ]:
with open('messages.json') as json_file:
    messages = json.load(json_file)

messages.pop()

with open('messages.json', 'w') as outfile:
    json.dump(messages, outfile)

In [ ]:
import re
import json
from transformers import pipeline

def update_messages(content, role, messages):
    new_message = {"role": role, "content": content}
    messages.append(new_message)
    with open('messages.json', 'w') as outfile:
        json.dump(messages, outfile)
    
    print(f"Step: {content}")
    return messages

def gpt4_agent(learn_dir=False):
    with open('messages.json') as json_file:
        messages = json.load(json_file)

    while True:
        response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=messages
        )

        response_content = response["choices"][0]["message"]["content"]
        
        messages = update_messages(response_content, "assistant", messages)
        
        # Define the regular expression pattern
        pattern = r'```bash\n(.*?)\n```'

        matches = re.findall(pattern, response_content, re.DOTALL)

        python_content = re.search(r'```python\n(.*?)\n```', response_content)

        python_prompt = "Write bash commands to implement those changes in the Python files."

        if matches:
        # matches is now a list of all bash commands in the string
            for bash_command in matches:
                stream = os.popen(bash_command)
                output = stream.read()
            
            if learn_dir:
                content = output
            
            elif python_content:
                content = python_prompt

            else:
                content = "Ok, did that"
        
        elif python_content:
            content = python_prompt

        else:
            learn_dir = False
            classifier = pipeline("zero-shot-classification")
            labels = ["suggestion for what to do next", "question"]
            results = classifier(sequences=response_content, candidate_labels=labels, hypothesis_template="This text is a {}")

            if results["scores"][0] > 0.70:
                if results["labels"][0] == "suggestion for what to do next":
                    content = "Create bash commands that implement those changes. Give me them one by one."
                
                else:
                    content = "My goal is to improve the model architecture of Alpaca-LoRA to make it a more powerful language model. Find the answer to that question in that context. If you can't, try another step in improving the language model."
            
        
        messages = update_messages(content, "user", messages)

In [ ]:
gpt4_agent(True)

In [ ]:
!pip install huggingface_hub

In [ ]:
os.environ["HUGGINGFACE_TOKEN"] = keys["hugging_face"]
!huggingface-cli login --token $HUGGINGFACE_TOKEN

In [ ]:
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj="messages.json",
    path_in_repo="messages.json",
    repo_id="jacksonkarel/selfmodifai-mpt-7b"
)